# From Movie Reviews to Box Office
Yen-Ting Chen

## Overview
This project aims to find the correlation between movie box office and movie reviews from Amazon.com. Machine learning models are fitted and reviews used to project a movie's box office.

There are three main parts to this project, including 1) data scraping and cleaning, 2) feature extraction and preprocessing, and 3) prediction modeling. Parts of the codes used are shown here, while the rest can be accessed on my [Github repository](https://github.com/janie128/Project-movies).

## 1) Data Scraping and Cleaning
There are two main data sets used in this project. One is the data set of movies and their box offices<sup>1</sup>, and the second is Amazon review data with their metadata<sup>2-4</sup>. 

#### Box office data
Box office data was scraped from the website for all years and movies.

In [ ]:
# The following URL and CSS templates rely on the layout of the website. They may change over time. 
pageUrlTemplate <- "http://www.boxofficemojo.com/daily/?view=bymovie&yr=%s&page=%d&sort=title&order=ASC"
years <- c("2015", "2014", "2013", "2012", "2011",
          "2010", "2009", "2008", "2007", "2006",
          "2005", "2004", "2003", "2002", "pre2002")
remainingPageLinkCss <- "font:nth-child(4) a" # how many page remaining
movieTitleCss <- "tr+ tr td:nth-child(1) font" # movie title
releaseGrossCss <- "tr+ tr td:nth-child(4) font" # release gross
releaseDateCss <- "tr+ tr td:nth-child(5) font" # release date

# Function for parsing the html page and extracting data of interest.
extractBoxOfficeFn <- function(page, boxOffice) {
  extractedTitles <- html_text(html_nodes(page, movieTitleCss))
  extractedReleaseDates <- html_text(html_nodes(page, releaseDateCss))
  # Extract release gross, strip "$" and ",", and cast to numeric.
  extractedReleaseGrosses <- html_text(html_nodes(page, releaseGrossCss))
  extractedReleaseGrosses <- gsub("\\$", "", extractedReleaseGrosses)
  extractedReleaseGrosses <- gsub(",", "", extractedReleaseGrosses)
  extractedReleaseGrosses <- as.numeric(extractedReleaseGrosses)

  extractedFromPage <- data.frame(
      title = extractedTitles,
      gross = extractedReleaseGrosses,
      releaseDate = extractedReleaseDates)
  return(extractedFromPage)
}

for (year in years) {
  pageUrl <- sprintf(pageUrlTemplate, year, 1);
  page <- read_html(pageUrl)
  numRemainingPages <- length(html_text(html_nodes(page, remainingPageLinkCss)))

  boxOffice <- rbind(boxOffice, extractBoxOfficeFn(page, boxOffice))
  for (pageIndex in 1:numRemainingPages) {
    pageUrl <- sprintf(pageUrlTemplate, year, pageIndex + 1);
    page <- read_html(pageUrl)
    boxOffice <- rbind(boxOffice, extractBoxOfficeFn(page, boxOffice))
  }
}

The data was then filtered to include data only after May 1996, to match the Amazon review data period. Initially, the box office amounts were to be adjusted to account for inflation or cultural factors (such as perhaps higher movie-going culture), by fitting a general (upwards) trend regression analysis and factoring it out. However, over the time period, an increase in the percentage of small films (lower-grossing) disrupts any such trend. This is evident in Fig. 1, where the percentage of movies grouped by their box office amounts ($log_{10}(gross)$) over the years is shown.  

![](./figures/box_office_years.png)
**Fig.1** Box office ($log_{10}(gross)$) sorted into 4 buckets and plotted over years. Percentage of highest grossing movies decreases while medium and lower grossing movies are on the rise.

Therefore, US inflation rates from 1995-2014 were obtained<sup>5</sup>, and was the only factor used to adjust the gross amount.

#### Amazon movie review data
Amazon "Movies & TV" review data and their metadata files were download in JSON format, which were further parsed and output with a Python script into strict JSON format for parsing in R. Review/metadata files were labeled with product titles as opposed to specifically movie titles, and matching was required between the movie titles from the box office data set and the product titles available in the metadata file. The data matching process involved stripping punctuation, converting to lower case, removing leading "the" for a more comprehensive matching, and pattern matching with regular expressions.  
  
The title-matched data was further cleaned and unnecessary variables removed. As some mismatching was unavoidable with extremely short titles, a subset of this data where the number of characters was greater than 4 was used for the rest of the analysis.

## 2) Feature Extraction and Preprocessing
Raw variables of interest available in the review data included the movie reviews, and overall rating (score) for each review. Further meaningful features must be extracted from this data.  
(1) Review count (by movie): This was extracted using the group_by(), summarize() and count() functions.  
(2) Movie average score: This was extracted using the group_by(), summarize() and mean() functions.  
(3) 

## References
1. [Box Office Mojo](http://www.boxofficemojo.com/daily/?view=bymovie&yr=all&sort=title&order=ASC&p=.htm)
2. [Dr. Julian McAuley's Amazon product data](http://jmcauley.ucsd.edu/data/amazon/)
3. "Image-based recommendations on styles and substitutes" J. McAuley, C. Targett, J. Shi, A. van den Hengel; *SIGIR*, 2015
4. "Inferring networks of substitutable and complementary products" J. McAuley, R. Pandey, J. Leskovec; *Knowledge Discovery and Data Mining*, 2015
5. [US Inflation Calculator](http://www.usinflationcalculator.com/inflation/historical-inflation-rates/)